<a href="https://colab.research.google.com/github/wenxuan0923/My-notes/blob/master/super_inheritance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `super()` inheritance in Python

In this note I will use lots of example to demonstrate the main concepts in Inheritance. Specifically, the following concepts will be covered:

- Singel Inheritance

- Use `super()` to access methods from superclasses

- Multiple Inheritance

- Method Resolution Order (MRO)

- Use `**kwargs` for more flexible class initialization

**Before jump into the details, recall that:**

- The primary use case of **Inheritance** is to extend the functionality of the inherited class (superclass) without repeating code.

- The `__init__()` function is for initialization. It is called automatically every time a class is being used to create a new object.

## Single Inheritance

• When using inheritance, if you are not redefining the `__init__` method in the subclass, the one of its parent class will be called automatically.

• Subclass will inherit the properties and methods from its parient class.

In [1]:
class SchoolMember:
  def __init__(self, name, age):
    self.name = name
    self.age = age
    
  def get_name(self):
    print(self.name)

  def get_age(self):
    print(self.age)
 
# The pass keyword means no other properties or methods is added to the class 
class Teacher(SchoolMember):
    pass     

In [2]:
hung = Teacher('Hung', 28)
hung.get_name()
hung.get_age()

Hung
28


• If the `__init__` in redefined in the subclass, the `__init__` from parent class will not be automatically invoked, but the subclass still have access to the methods defined in its parent class

• To keep the inheritance of the parent's `__init__()` function, we need to add a call to the parent's `__init__()` function.


In [18]:
class SchoolMember:
  def __init__(self, name, age):
    self.name = name
    self.age = age
    
  def sayhi(self):
    print('Hi! this is SchoolMemer')
 
class Teacher(SchoolMember):
  def __init__(self, name, age, subject):
    self.subject = subject

  def get_subject(self):
    print(self.subject)

$\color{green}{\text{✔ This works:}}$

In [19]:
hung = Teacher('Hung', 28, 'Math')
hung.sayhi()
hung.get_subject()

Hi! this is SchoolMemer
Math


$\color{red}{\text{✖ This does not work:}}$

In [20]:
print(hung.name)
print(hung.age)

AttributeError: ignored

This command leads to $\color{indianred}{\text{AttributeError}}$ since the `__init__()` from the parent class SchoolMember has never been called. 

To fix this, you need to explicitly call the `__init__()` of the parent class inside the init of the subclass. 

In [26]:
class SchoolMember:
  def __init__(self, name, age):
    self.name = name
    self.age = age
 
class Teacher(SchoolMember):
  def __init__(self, name, age, subject):
    SchoolMember.__init__(self, name, age)
    self.subject = subject

In [27]:
hung = Teacher('Hung', 28, 'Math')
print(hung.name)
print(hung.age)
print(hung.subject)

Hung
28
Math


**Using `super()` to call the `__init__` of the parent class**

Instead of using `SchoolMember.__init__(self, name, age)`, we can use Python `super()` function which allows us to refer the superclass implicitly, meaning we don’t need to write the name of superclass explicitly. 

• This `super()` alone returns a temporary object (called **roxy object**) of the superclass that then allows you to call that superclass’s methods. 

• Note we don't need the `self` argument when calling method from `super()`.

The following code will do the same thing with the one above. 

In [24]:
class SchoolMember:
  def __init__(self, name, age):
    self.name = name
    self.age = age
 
class Teacher(SchoolMember):
  def __init__(self, name, age, subject):
    super().__init__(name, age)
    self.subject = subject

In [25]:
hung = Teacher('Hung', 28, 'Math')
print(hung.name)
print(hung.age)
print(hung.subject)

Hung
28
Math


**We can also use super() to call any other method from its parent class**

In [28]:
class SchoolMember:
  def __init__(self, name, age, gender):
    self.name = name
    self.age = age
    self.gender = gender

  def sayhi(self):
    print('Hi! This is SchoolMemer')
    
  def get_portrait(self):
    return '👦' if self.gender=='male' else '👧'

class Teacher(SchoolMember):
  def __init__(self, name, age, gender, subject):
    super().__init__(name, age, gender)
    self.subject = subject

  def sayhi(self):
    super().sayhi()
    profile = super().get_portrait()
    print('Hi! This is {} Teacher {} {}'.format(self.subject, self.name, profile))

In [29]:
hung = Teacher('Hung', 28, 'male', 'Math')
hung.sayhi()

Hi! This is SchoolMemer
Hi! This is Math Teacher Hung 👦


## Multiple Inheritance

Python supports multiple inheritance, in which a subclass can inherit from multiple superclasses.  

Let's now define a class **Myclass** which inherit from two superclass: **Base1** and **Base2**.

In [30]:
class Base1:
  def __init__(self):
    print('Initializing Base 1')

  def sayhi(self):
    print('Hi from Base 1')  

class Base2:
  def __init__(self):
    print('Initializing Base 2')

  def sayhi(self):
    print('Hi from Base 2')  

class Myclass(Base1, Base2):
  def __init__(self):
    super().__init__()
    
  def sayhi(self):
    print('Hi from Myclass')
    super().sayhi()

In [31]:
myObject = Myclass()
myObject.sayhi()

Initializing Base 1
Hi from Myclass
Hi from Base 1


You may notice that even though both superclasses have `__init__()` and `sayhi()` methods, only those from Base1 got invoked when calling these methods using `super()`. This is because of something called **Method Resolution Order (MRO)**. It tells Python where to search for methods when being called using `super()`. You can easily inspect the order by checking the `__mro__` attribute of the class.

In [32]:
Myclass.__mro__

(__main__.Myclass, __main__.Base1, __main__.Base2, object)

This tells us that methods will be searched in the order:<strong> Myclass $\Rightarrow$ Base1 $\Rightarrow$ Base2 </strong>. 

What if we swich the position of Base1 and Base2?

In [33]:
class Base1:
  def __init__(self):
    print('Initializing Base 1')

  def sayhi(self):
    print('Hi from Base 1')  

class Base2:
  def __init__(self):
    print('Initializing Base 2')

  def sayhi(self):
    print('Hi from Base 2')  

class Myclass(Base2, Base1):
  def __init__(self):
    print('Initializing Myclass')
    super().__init__()
    
  def sayhi(self):
    print('Hi from Myclass')
    super().sayhi()

In [34]:
Myclass.__mro__

(__main__.Myclass, __main__.Base2, __main__.Base1, object)

In [35]:
myObject = Myclass()
myObject.sayhi()

Initializing Myclass
Initializing Base 2
Hi from Myclass
Hi from Base 2


In this case, only methods from Base2 got called, and the MRO has changed accordingly.
 
Let's check another example where Myclass inherit from Base2, and Base2 inherit from Base1:

In [101]:
class Base1:
  def __init__(self):
    print('Initializing Base 1')

  def sayhi(self):
    print('Hi from Base 1')  

class Base2(Base1):
  def __init__(self):
    print('Initializing Base 2')
    super().__init__()

  def sayhi(self):
    print('Hi from Base 2') 
    super().__init__() 

class Myclass(Base2):
  def __init__(self):
    print('Initializing Myclass')
    super().__init__()
    
  def sayhi(self):
    print('Hi from Myclass')
    super().sayhi()

In [103]:
Myclass.__mro__

(__main__.Myclass, __main__.Base2, __main__.Base1, object)

In [102]:
myObject = Myclass()
myObject.sayhi()

Initializing Myclass
Initializing Base 2
Initializing Base 1
Hi from Myclass
Hi from Base 2
Initializing Base 1


the `__init__()` function of Myclass got called at first, then class Base2 and after that Base1. Similar thing happened by calling sayhi() method.

### More flexible `__init__` with **\*\*kwargs**


\*\*kwargs allows users of these objects to instantiate them only with the arguments that make sense for that particular object.

In [ ]:
class A:
  def __init__(self, a, **kwargs):
    self.a = a
    self.c = 5

class B(A):
  def __init__(self, b, **kwargs):
    self.b = b
    super().__init__(**kwargs)

In [226]:
b1 = B(a=1, b=3, c=7)
print(b1.b)
print(b1.a)
print(b1.c)

3
1
5


Things can get a little confusing when it comes to multiple inheritence.

In [280]:
class A:
  def __init__(self, a):
    self.a = a

class B:
  def __init__(self, b):
    self.b = b

class C(A, B):
  def __init__(self, c, **kwargs):
    self.c = c
    super().__init__(**kwargs)

In [281]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

$\color{green}{\text{✔ This works:}}$

In [282]:
c1 = C(c=7, a=1)
print(c1.c)
print(c1.a)

7
1


$\color{red}{\text{✖ This does not work:}}$

In [283]:
c2 = C(c=7, a=1, b=2)
print(c2.c)
print(c2.b)

TypeError: ignored

This is because the `super(MyClass, self).__init__()` provides the next `__init__()` method according to the Method Resolution Ordering (MRO) in the context of the complete inheritance hierarchy. 

In thie example above, only class C has `super()`, the execution stops at the end of `A.__init__()` and never make it to `B.__init__()`.


To fix this, we just need to add `super().__init__()` to both A and B.

In [284]:
class A:
  def __init__(self, a, **kwargs):
    self.a = a
    super().__init__(**kwargs)

class B:
  def __init__(self, b, **kwargs):
    self.b = b
    super().__init__(**kwargs)

class C(A, B):
  def __init__(self, c, **kwargs):
    self.c = c
    super().__init__(**kwargs)

In [285]:
c2 = C(a=1, c=7, b=2)
print(c2.c)
print(c2.b)
print(c2.a)

7
2
1


I find this concept is better illustrated by the example below, which is originally from <a href='https://stackoverflow.com/questions/3277367/how-does-pythons-super-work-with-multiple-inheritance' target='_blank'>StackOverFlow</a>.

In [275]:
class First:
  def __init__(self):
    print("First(): entering")
    super().__init__()
    print("First(): exiting")

class Second:
  def __init__(self):
    print("Second(): entering")
    super().__init__()
    print("Second(): exiting")

class Third(First, Second):
  def __init__(self):
    print("Third(): entering")
    super().__init__()
    print("Third(): exiting")

In [276]:
Third.__mro__

(__main__.Third, __main__.First, __main__.Second, object)

In [277]:
Third()

Third(): entering
First(): entering
Second(): entering
Second(): exiting
First(): exiting
Third(): exiting


The MRO to resolve `Third.__init__()` is calculated: <strong>Third $\Rightarrow$ First $\Rightarrow$ Second $\Rightarrow$ Object.</strong>  According to MRO:


1. `Third.__init__()` executes first:
> - prints `Third(): entering`
> - then `super(Third, self).__init__()` executes and MRO returns `First.__init__()` which is called next.

2. `First.__init__()` executes:
> prints `First(): entering`
>
> then `super(First, self).__init__()` executes and MRO returns `Second.__init__()` which is called next.

3. `Second.__init__()` executes:
> - prints `Second(): entering`
>
> - then `super(Second, self).__init__()` executes and MRO returns `object.__init__()` which is called next.

4. `object.__init__()` executes (no print statements in the code there)

5. execution goes back to `Second.__init__()` which then prints `Second(): exiting`

6. execution goes back to `First.__init__()` which then prints `First(): exiting`

7. execution goes back to `Third.__init__()` which then prints `Third(): exiting`

### It can become very powerful when being used together with **\*\*kwargs**:

In [286]:
class A:
  def __init__(self, a, **kwargs):
    self.a = a
    super().__init__(**kwargs)
  def heart(self):
    return '❤'

class B:
  def __init__(self, b, **kwargs):
    self.b = b
    super().__init__(**kwargs)
  def star(self):
    return '⭐'

class C(A, B):
  def __init__(self, c, **kwargs):
    self.c = c
    kwargs['a'] = c - 1
    kwargs['b'] = c + 1 
    super().__init__(**kwargs)
  

In [287]:
myVar = C(3)

In [288]:
print(myVar.a)
print(myVar.b)
print(myVar.c)
print(myVar.heart())
print(myVar.star())

2
4
3
❤
⭐


### A little more complicated example

In this example we will calculate the area of RightPyramid (a pyramid with a square base) illustrated below. This example is originally from: <a target='_blank' href='https://realpython.com/python-super/#super-in-multiple-inheritance'>Supercharge Your Classes With Python super() </a>. I made some modifications to simplify the code a little bit.

<center><img src='https://drive.google.com/uc?id=1IB8CgbAQtrczR_aio3d0XwOrH248OZtP'></img></center>

In this exampe we try to call both `.area()` from parent class: Square and `tri_area()` function from another parent class Triangle using `super()` inside subclass RightPyramid. Methods `area()` and `area2()` in RightPyramid class are two different ways to calculte the area. They should yield to the same result.

In [37]:
class Square:
  def __init__(self, length, **kwargs):
    self.length = length
    super().__init__(**kwargs)
  
  def area(self):
    return self.length * self.length

  def perimeter(self):
    return 4 * self.length

class Triangle:
  def __init__(self, base, height, **kwargs):
    self.base = base
    self.height = height
    super().__init__(**kwargs)

  def tri_area(self):
    return 0.5 * self.base * self.height
  
  def hi(self):
    print('Hi from Triangle')
     
class RightPyramid(Square, Triangle):
  def __init__(self, base, slant_height, **kwargs):
    self.base = base
    self.slant_height = slant_height
    kwargs["height"] = slant_height
    kwargs["length"] = base
    super().__init__(base=base, **kwargs)

  def area(self):
    base_area = super().area()
    perimeter = super().perimeter()
    return 0.5 * perimeter * self.slant_height + base_area

  def area_2(self):
    base_area = super().area()
    triangle_area = super().tri_area()
    return triangle_area * 4 + base_area

  def sayhi(self):
    super().hi()

In [38]:
pyramid = RightPyramid(2, 4)
print(pyramid.area())
print(pyramid.area_2())
print(pyramid.base)
print(pyramid.height)
print(pyramid.length)
pyramid.hi()

20.0
20.0
2
4
2
Hi from Triangle
